In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os
from utils.common import (
    m2f_dataset_collate,
    m2f_extract_pred_maps_and_masks,
    set_seed,
    pixel_mean_std,
    CADIS_PIXEL_MEAN,
    CADIS_PIXEL_STD,
    CAT1K_PIXEL_MEAN,
    CAT1K_PIXEL_STD
)
from utils.dataset_utils import (
    get_cadisv2_dataset,
    get_cataract1k_dataset,
    ZEISS_CATEGORIES,
)
from utils.medical_datasets import Mask2FormerDataset
from transformers import (
    Mask2FormerForUniversalSegmentation,
    SwinModel,
    SwinConfig,
    Mask2FormerConfig,
    AutoImageProcessor,
    Mask2FormerImageProcessor
)
from torch.utils.data import DataLoader
import evaluate
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from dotenv import load_dotenv
import wandb
from copy import deepcopy
import shutil
import random
from utils.altered_m2f import CustomMask2Former
from utils.wandb_utils import log_table_of_images

/notebooks/venv2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#set_seed(42) # seed everything

In [4]:
NUM_CLASSES = len(ZEISS_CATEGORIES) - 3  + 1# Remove class incremental and add background!!!
SWIN_BACKBONE = "microsoft/swin-tiny-patch4-window7-224"#"microsoft/swin-large-patch4-window12-384"

# Download pretrained swin model
swin_model = SwinModel.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)
swin_config = SwinConfig.from_pretrained(
    SWIN_BACKBONE, out_features=["stage1", "stage2", "stage3", "stage4"]
)

# Create Mask2Former configuration based on Swin's configuration
mask2former_config = Mask2FormerConfig(
    backbone_config=swin_config, num_labels=NUM_CLASSES #, ignore_value=BG_VALUE
)

# Create the Mask2Former model with this configuration
model = Mask2FormerForUniversalSegmentation(mask2former_config)

# Reuse pretrained parameters
for swin_param, m2f_param in zip(
    swin_model.named_parameters(),
    model.model.pixel_level_module.encoder.named_parameters(),
):
    m2f_param_name = f"model.pixel_level_module.encoder.{m2f_param[0]}"

    if swin_param[0] == m2f_param[0]:
        model.state_dict()[m2f_param_name].copy_(swin_param[1])
        continue

    print(f"Not Matched: {m2f_param[0]} != {swin_param[0]}")

/notebooks/venv2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Not Matched: hidden_states_norms.stage1.weight != layernorm.weight
Not Matched: hidden_states_norms.stage1.bias != layernorm.bias


In [5]:
# Helper function to load datasets
def load_dataset(dataset_getter, data_path, domain_incremental):
    return dataset_getter(data_path, domain_incremental=domain_incremental)


# Helper function to create dataloaders for a dataset
def create_dataloaders(
    dataset, batch_size, shuffle, num_workers, drop_last, pin_memory, collate_fn,generator
):
    return {
        "train": DataLoader(
            dataset["train"],
            batch_size=batch_size,
            shuffle=shuffle,
            num_workers=num_workers,
            drop_last=drop_last,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
            generator=g
        ),
        "val": DataLoader(
            dataset["val"],
            batch_size=batch_size*2,
            shuffle=shuffle,
            num_workers=num_workers,
            drop_last=drop_last,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
            generator=g
        ),
        "test": DataLoader(
            dataset["test"],
            batch_size=batch_size*2,
            shuffle=False,
            num_workers=num_workers,
            drop_last=False,
            pin_memory=pin_memory,
            collate_fn=collate_fn,
            generator=g
        ),
    }


# Load datasets
datasets = {
    "A": load_dataset(get_cadisv2_dataset, "../../storage/data/CaDISv2", True),
    "B": load_dataset(get_cataract1k_dataset, "../../storage/data/cataract-1k", True),
}

#pixel_mean_A,pixel_std_A=pixel_mean_std(datasets["A"][0])
#print("pixel mean for A:",pixel_mean_A,"pixel std",pixel_std_A)
pixel_mean_A=np.array(CADIS_PIXEL_MEAN)
pixel_std_A=np.array(CADIS_PIXEL_STD)

#pixel_mean_B,pixel_std_B=pixel_mean_std(datasets["B"][0])
#print("pixel mean for B:",pixel_mean_B,"pixel std",pixel_std_B)
pixel_mean_B=np.array(CAT1K_PIXEL_MEAN)
pixel_std_B=np.array(CAT1K_PIXEL_STD)


# Define preprocessor
swin_processor = AutoImageProcessor.from_pretrained(SWIN_BACKBONE)
m2f_preprocessor_A = Mask2FormerImageProcessor(
    reduce_labels=False,
    ignore_index=255,
    do_resize=False,
    do_rescale=False,
    do_normalize=True,
    image_std=pixel_std_A,
    image_mean=pixel_mean_A,
)

m2f_preprocessor_B = Mask2FormerImageProcessor(
    reduce_labels=False,
    ignore_index=255,
    do_resize=False,
    do_rescale=False,
    do_normalize=True,
    image_std=pixel_std_B,
    image_mean=pixel_mean_B,
)

# Create Mask2Former Datasets
m2f_datasets = {
    "A": {
        "train": Mask2FormerDataset(datasets["A"][0], m2f_preprocessor_A),
        "val": Mask2FormerDataset(datasets["A"][1], m2f_preprocessor_A),
        "test": Mask2FormerDataset(datasets["A"][2], m2f_preprocessor_A),
    },
    "B": {
        "train": Mask2FormerDataset(datasets["B"][0], m2f_preprocessor_B),
        "val": Mask2FormerDataset(datasets["B"][1], m2f_preprocessor_B),
        "test": Mask2FormerDataset(datasets["B"][2], m2f_preprocessor_B),
    },
}

metric = evaluate.load("mean_iou")
# DataLoader parameters
N_WORKERS = 4
BATCH_SIZE = 8
SHUFFLE = True
DROP_LAST = True

seed=42
g = torch.Generator()
g.manual_seed(seed)

dataloader_params = {
    "batch_size": BATCH_SIZE,
    "shuffle": SHUFFLE,
    "num_workers": N_WORKERS,
    "drop_last": DROP_LAST,
    "pin_memory": True,
    "collate_fn": m2f_dataset_collate,
    "generator":g
}

# Create DataLoaders
dataloaders = {
    key: create_dataloaders(m2f_datasets[key], **dataloader_params)
    for key in m2f_datasets
}

print(dataloaders)

/notebooks/venv2/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


{'A': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f4ca02b02c0>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7f4ca0675610>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f4ca0457aa0>}, 'B': {'train': <torch.utils.data.dataloader.DataLoader object at 0x7f4dc2fb92e0>, 'val': <torch.utils.data.dataloader.DataLoader object at 0x7f4dc2fb9580>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x7f4dc2fb9250>}}


In [6]:
m2f_preprocessor_A.reduce_labels, m2f_preprocessor_A.ignore_index

(False, 255)

In [7]:
# Check if CUDA is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
BG_VALUE_255=255
base_run_name="M2F-Swin-Tiny-Train_Cadis"
new_run_name="M2F-Swin-Tiny-Latent-Replay32MB"
project_name = "M2F_latest"
user_or_team = "continual-learning-tum"

Using device: cuda


In [8]:
# Tensorboard setup
out_dir="outputs/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
if not os.path.exists(out_dir+"runs"):
    os.makedirs(out_dir+"runs")
%load_ext tensorboard
%tensorboard --logdir outputs/runs

In [9]:
!CUDA_LAUNCH_BLOCKING=1

In [10]:
# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")

# Model checkpointing
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{base_run_name}/best_model/"
if not os.path.exists(best_model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{base_run_name}/final_model/"
if not os.path.exists(final_model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

In [11]:
# WandB for team usage !!!!

#wandb.login() # use this one if a different person is going to run the notebook
wandb.login(relogin=False) # if the same person in the last run is going to run the notebook again


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ge85ket (continual-learning-tum). Use `wandb login --relogin` to force relogin


True

# First train on dataset A

## Test results on A

In [12]:

# Construct the artifact path
artifact_path = f"{user_or_team}/{project_name}/best_model_{base_run_name}:latest"

metric = evaluate.load("mean_iou")
# Load from W&B
api = wandb.Api()
artifact=api.artifact(artifact_path)
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, f"best_model_{base_run_name}.pth" )
model_state_dict = torch.load(model_state_dict_path)
model = Mask2FormerForUniversalSegmentation(mask2former_config)
model.load_state_dict(model_state_dict)
model.to(device)

wandb: Downloading large artifact best_model_M2F-Swin-Tiny-Train_Cadis:latest, 181.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.8


Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [15]:
model.eval()
test_running_loss = 0
CURR_TASK="A"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")

BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_A
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_A, # remove normalization
                pixel_std_A, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1
# Log table
wandb.log({f"{CURR_TASK}_TEST_AFTER_TRAINING_A": table})

# After compute the batches that were added are deleted
test_metrics_A = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_A["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
"""wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})"""
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")

Test loop: 100%|██████████| 37/37 [01:25<00:00,  2.31s/it, loss=79.0944] 


Test Loss: 16.4511, Test mIoU: 0.7464


## Test results on B

In [16]:
model.eval()
test_running_loss = 0
CURR_TASK="B"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")

BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_B, # remove normalization
                pixel_std_B, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1

# Log table
wandb.log({f"{CURR_TASK}_TEST_AFTER_TRAINING_A": table})

# After compute the batches that were added are deleted
test_metrics_B_before = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_B_before["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
"""wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})"""
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")
#wandb.finish()

Test loop: 100%|██████████| 15/15 [00:36<00:00,  2.40s/it, loss=126.2300]


Test Loss: 41.0157, Test mIoU: 0.2717


# Now train on B and forget A

In [7]:
"""
input_0=m2f_datasets["A"]["train"][0]
input_0["pixel_values"].size(),input_0["pixel_mask"].size(),input_0["mask_labels"][0].size(),input_0["class_labels"][0].size()
"""

(torch.Size([1, 3, 270, 480]),
 torch.Size([1, 270, 480]),
 torch.Size([3, 270, 480]),
 torch.Size([3]))

In [23]:
#414720
#latent_vector.numel(),latent_vector.element_size(),latent_vector.size()

(103680, 4, torch.Size([1, 768, 9, 15]))

## Sample latent vectors

In [17]:
model.eval()
latent_vectors=[]
pixel_mask=[]
mask_labels=[]
class_labels=[]
curr_byte_size=0

# Target size in bytes (32MB)
target_size_bytes = 32 * 1024 * 1024
N=21

# Calculate the byte size of one bottleneck vector
def calculate_sample_size(bottleneck_vector):
    bottleneck_byte_size = bottleneck_vector.numel() * bottleneck_vector.element_size()  # Number of elements * bytes per element
    return bottleneck_byte_size


indices = list(range(len(m2f_datasets["A"]["train"])))
random.shuffle(indices)

with torch.no_grad():
    for i in range(N):
        idx=indices[i]
        inputs = m2f_datasets["A"]["train"][idx]
        inputs["pixel_values"] = inputs["pixel_values"].to(device)
        inputs["pixel_mask"] = inputs["pixel_mask"].to(device)
        inputs["mask_labels"] = [entry.to(device) for entry in inputs["mask_labels"]]
        inputs["class_labels"] = [entry.to(device) for entry in inputs["class_labels"]]

        output=model(**inputs,output_hidden_states=True)
        backbone_vectors=output.encoder_hidden_states
        latent_vectors.append(backbone_vectors)
        mask_labels.append(inputs["mask_labels"][0])
        class_labels.append(inputs["class_labels"][0])
        pixel_mask.append(inputs["pixel_mask"])
        
num_replay_samples_per_batch = 8

In [21]:
len(latent_vectors)

21

## Train

In [18]:
# Construct the artifact path
artifact_path = f"{user_or_team}/{project_name}/best_model_{base_run_name}:latest"

metric = evaluate.load("mean_iou")
# Load from W&B
api = wandb.Api()
artifact=api.artifact(artifact_path)
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, f"best_model_{base_run_name}.pth" )
model_state_dict = torch.load(model_state_dict_path)
model = CustomMask2Former(mask2former_config) # !!!!!!!!!!!
model.load_state_dict(model_state_dict)
model.to(device)

wandb: Downloading large artifact best_model_M2F-Swin-Tiny-Train_Cadis:latest, 181.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.5


CustomMask2Former(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value): Linear(in_featu

In [19]:
# Training
NUM_EPOCHS = 200
LEARNING_RATE = 1e-4
LR_MULTIPLIER = 0.1
BACKBONE_LR = 0 # Freeze learning for the backbone
WEIGHT_DECAY = 0.05
PATIENCE=15
model.train()
metric = evaluate.load("mean_iou")

for name, param in model.named_parameters(): #Freeze backbone
    if name.startswith("model.pixel_level_module.encoder"):
        param.requires_grad = False
    
encoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.encoder")
]
decoder_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.pixel_level_module.decoder")
]
transformer_params = [
    param
    for name, param in model.named_parameters()
    if name.startswith("model.transformer_module")
]
class_prediction_params=[
    param
    for name, param in model.named_parameters() 
    if not name.startswith("model.pixel_level_module.encoder") and not name.startswith("model.transformer_module") and not name.startswith("model.pixel_level_module.decoder")
]
optimizer = optim.AdamW(
    [
        {"params": encoder_params, "lr": BACKBONE_LR},
        {"params": decoder_params},
        {"params": transformer_params},
        {"params": class_prediction_params}
    ],
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
)

scheduler = optim.lr_scheduler.PolynomialLR(
    optimizer, total_iters=NUM_EPOCHS, power=0.9
)

In [14]:
wandb.init(
    project=project_name,
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_multiplier": LR_MULTIPLIER,
        "backbone_learning_rate": BACKBONE_LR,
        "learning_rate_scheduler": scheduler.__class__.__name__,
        "optimizer": optimizer.__class__.__name__,
        "backbone": SWIN_BACKBONE,
        "m2f_preprocessor": m2f_preprocessor_B.__dict__,
        "m2f_model_config": model.config
    },
    name=new_run_name,
    notes="M2F with tiny Swin backbone pretrained on ImageNet-1K. \
        Scenario: Pretrained on A, Train on B, Test forgetting on A"
)

print("wandb run id:",wandb.run.id)

# Tensorboard logging
writer = SummaryWriter(log_dir=out_dir + "runs")
# Model checkpointing
model_dir = out_dir + "models/"
if not os.path.exists(model_dir):
    print("Store weights in: ", model_dir)
    os.makedirs(model_dir)

best_model_dir = model_dir + f"{new_run_name}/best_model/"
if not os.path.exists(best_model_dir):
    print("Store best model weights in: ", best_model_dir)
    os.makedirs(best_model_dir)
final_model_dir = model_dir + f"{new_run_name}/final_model/"
if not os.path.exists(final_model_dir):
    print("Store final model weights in: ", final_model_dir)
    os.makedirs(final_model_dir)

wandb run id: 979sl60u


In [ ]:
# Save the preprocessor
m2f_preprocessor_B.save_pretrained(model_dir + new_run_name)

In [20]:
# To avoid making stupid errors
CURR_TASK = "B"

# For storing the model
best_val_metric = -np.inf
best_model_weights=None # best model weights are stored here


# Move model to device
model.to(device)
counter=0
for epoch in range(NUM_EPOCHS):
    model.train()
    train_running_loss = 0.0
    val_running_loss = 0.0

    # Set up tqdm for the training loop
    train_loader = tqdm(
        dataloaders[CURR_TASK]["train"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Training"
    )

    for batch in train_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]

        # Compute output and loss
        sampled_replay_indices = random.sample(range(len(latent_vectors)), num_replay_samples_per_batch)
        old_latent_vectors_1=torch.cat([latent_vectors[i][0] for i in sampled_replay_indices],dim=0).to(device)
        old_latent_vectors_2=torch.cat([latent_vectors[i][1] for i in sampled_replay_indices],dim=0).to(device)
        old_latent_vectors_3=torch.cat([latent_vectors[i][2] for i in sampled_replay_indices],dim=0).to(device)
        old_latent_vectors_4=torch.cat([latent_vectors[i][3] for i in sampled_replay_indices],dim=0).to(device)
        old_latent_vectors=(old_latent_vectors_1,old_latent_vectors_2,old_latent_vectors_3,old_latent_vectors_4)
        old_pixel_mask=torch.cat([pixel_mask[i] for i in sampled_replay_indices],dim=0).to(device)
        old_mask_labels=[mask_labels[i].to(device) for i in sampled_replay_indices]
        old_class_labels=[class_labels[i].to(device) for i in sampled_replay_indices]
        
        old_samples={"old_latent_vectors":old_latent_vectors,"old_pixel_mask":old_pixel_mask,"old_mask_labels":old_mask_labels,"old_class_labels":old_class_labels}
        
        outputs = model(**batch,**old_samples)

        loss = outputs.loss

        # Compute gradient and perform step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record losses
        current_loss = loss.item() * (batch["pixel_values"].size(0)+num_replay_samples_per_batch)
        train_running_loss += current_loss
        train_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)

    # After compute the batches that were added are deleted
    mean_train_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )["mean_iou"]

    # Validation phase
    model.eval()
    val_loader = tqdm(
        dataloaders[CURR_TASK]["val"], desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} Validation"
    )
    with torch.no_grad():
        for batch in val_loader:
            # Move everything to the device
            batch["pixel_values"] = batch["pixel_values"].to(device)
            batch["pixel_mask"] = batch["pixel_mask"].to(device)
            batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
            batch["class_labels"] = [
                entry.to(device) for entry in batch["class_labels"]
            ]
            # Compute output and loss
            outputs = model(**batch)

            loss = outputs.loss
            # Record losses
            current_loss = loss.item() * batch["pixel_values"].size(0)
            val_running_loss += current_loss
            val_loader.set_postfix(loss=f"{current_loss:.4f}")

            # Extract and compute metrics
            pred_maps, masks = m2f_extract_pred_maps_and_masks(
                batch, outputs, m2f_preprocessor_B
            )
            metric.add_batch(references=masks, predictions=pred_maps)

    # After compute the batches that were added are deleted
    mean_val_iou = metric.compute(
        num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
    )["mean_iou"]

    epoch_train_loss = train_running_loss / len(dataloaders[CURR_TASK]["train"].dataset)
    epoch_val_loss = val_running_loss / len(dataloaders[CURR_TASK]["val"].dataset)

    writer.add_scalar(f"Loss/train_{new_run_name}_{CURR_TASK}", epoch_train_loss, epoch + 1)
    writer.add_scalar(f"Loss/val_{new_run_name}_{CURR_TASK}", epoch_val_loss, epoch + 1)
    writer.add_scalar(f"mIoU/train_{new_run_name}_{CURR_TASK}", mean_train_iou, epoch + 1)
    writer.add_scalar(f"mIoU/val_{new_run_name}_{CURR_TASK}", mean_val_iou, epoch + 1)

    wandb.log({
        f"Loss/train_replay_A_{CURR_TASK}": epoch_train_loss,
        f"Loss/val_replay_A_{CURR_TASK}": epoch_val_loss,
        f"mIoU/train_replay_A_{CURR_TASK}": mean_train_iou,
        f"mIoU/val_replay_A_{CURR_TASK}": mean_val_iou
    })

    tqdm.write(
        f"Epoch {epoch + 1}/{NUM_EPOCHS}, Train Loss: {epoch_train_loss:.4f}, Train mIoU: {mean_train_iou:.4f}, Validation Loss: {epoch_val_loss:.4f}, Validation mIoU: {mean_val_iou:.4f}"
    )
    
    if mean_val_iou > best_val_metric:
        best_val_metric = mean_val_iou
        #model.save_pretrained(f"{best_model_dir}{CURR_TASK}/")
        best_model_weights = deepcopy(model.state_dict())
        counter=0
    else:
        counter+=1
        if counter == PATIENCE:
            print("Early stopping at epoch",epoch)
            break
            
os.makedirs(f"{best_model_dir}{CURR_TASK}/",exist_ok=True)
artifact = wandb.Artifact(f"best_model_{new_run_name}", type="model")
artifact.add_file(f"{best_model_dir}{CURR_TASK}/best_model_{new_run_name}.pth", torch.save(best_model_weights, f"{best_model_dir}{CURR_TASK}/best_model_{new_run_name}.pth"))
wandb.run.log_artifact(artifact)

if os.path.exists(model_dir + f"{new_run_name}"):
    shutil.rmtree(model_dir + f"{new_run_name}")

Epoch 1/200 Validation: 100%|██████████| 14/14 [00:31<00:00,  2.22s/it, loss=135.2564]


Epoch 1/200, Train Loss: 15.4555, Train mIoU: 0.7860, Validation Loss: 8.4713, Validation mIoU: 0.6585


Epoch 2/200 Validation: 100%|██████████| 14/14 [00:31<00:00,  2.27s/it, loss=133.4543]


Epoch 2/200, Train Loss: 11.0024, Train mIoU: 0.8626, Validation Loss: 7.8486, Validation mIoU: 0.6991


Epoch 3/200 Validation: 100%|██████████| 14/14 [00:28<00:00,  2.05s/it, loss=123.2820]


...

Epoch 38/200, Train Loss: 6.8619, Train mIoU: 0.9433, Validation Loss: 7.3371, Validation mIoU: 0.6792


Epoch 39/200 Validation: 100%|██████████| 14/14 [00:15<00:00,  1.12s/it, loss=99.3459] 


Epoch 39/200, Train Loss: 6.7378, Train mIoU: 0.9483, Validation Loss: 6.9862, Validation mIoU: 0.7351


Epoch 40/200 Validation: 100%|██████████| 14/14 [00:15<00:00,  1.08s/it, loss=100.5350]


Epoch 40/200, Train Loss: 6.5863, Train mIoU: 0.9482, Validation Loss: 7.0609, Validation mIoU: 0.7508
Early stopping at epoch 39


## Test results on B first

In [22]:
# Load best model and evaluate on test
#model = Mask2FormerForUniversalSegmentation.from_pretrained(f"{best_model_dir}{CURR_TASK}/").to(device)

# Construct the artifact path
artifact_path = f"{user_or_team}/{project_name}/best_model_{new_run_name}:latest"

# Load from W&B
api = wandb.Api()
artifact=api.artifact(artifact_path)
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, f"best_model_{new_run_name}.pth" )
model_state_dict = torch.load(model_state_dict_path)
model = Mask2FormerForUniversalSegmentation(mask2former_config)
model.load_state_dict(model_state_dict)
model.to(device)

wandb: Downloading large artifact best_model_M2F-Swin-Tiny-Latent-Replay32MB:latest, 181.31MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.2


Mask2FormerForUniversalSegmentation(
  (model): Mask2FormerModel(
    (pixel_level_module): Mask2FormerPixelLevelModule(
      (encoder): SwinBackbone(
        (embeddings): SwinEmbeddings(
          (patch_embeddings): SwinPatchEmbeddings(
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
          )
          (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): SwinEncoder(
          (layers): ModuleList(
            (0): SwinStage(
              (blocks): ModuleList(
                (0-1): 2 x SwinLayer(
                  (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attention): SwinAttention(
                    (self): SwinSelfAttention(
                      (query): Linear(in_features=96, out_features=96, bias=True)
                      (key): Linear(in_features=96, out_features=96, bias=True)
                      (value

In [23]:
model.eval()
test_running_loss = 0
CURR_TASK="B"
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")
BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_B
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_B, # remove normalization
                pixel_std_B, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1
# Log table
wandb.log({f"{CURR_TASK}_TEST_AFTER_TRAINING_B": table})

# After compute the batches that were added are deleted
test_metrics_B = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_B["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_{CURR_TASK}": final_test_loss,
    f"mIoU/test_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")

Test loop: 100%|██████████| 15/15 [00:20<00:00,  1.37s/it, loss=16.8909] 


Test Loss: 7.3802, Test mIoU: 0.8015


## Test results on A after training on B

In [24]:
# To avoid making stupid errors
CURR_TASK = "A"

model.eval()
test_running_loss = 0
test_loader = tqdm(dataloaders[CURR_TASK]["test"], desc="Test loop")
BATCH_INDEX = 0
table = wandb.Table(columns=["ID", "Image"])
with torch.no_grad():
    for batch in test_loader:
        # Move everything to the device
        batch["pixel_values"] = batch["pixel_values"].to(device)
        batch["pixel_mask"] = batch["pixel_mask"].to(device)
        batch["mask_labels"] = [entry.to(device) for entry in batch["mask_labels"]]
        batch["class_labels"] = [entry.to(device) for entry in batch["class_labels"]]
        # Compute output and loss
        outputs = model(**batch)

        loss = outputs.loss
        # Record losses
        current_loss = loss.item() * batch["pixel_values"].size(0)
        test_running_loss += current_loss
        test_loader.set_postfix(loss=f"{current_loss:.4f}")

        # Extract and compute metrics
        pred_maps, masks = m2f_extract_pred_maps_and_masks(
            batch, outputs, m2f_preprocessor_A
        )
        metric.add_batch(references=masks, predictions=pred_maps)
        if BATCH_INDEX <5:
            # Visualize
            log_table_of_images(
                table, # common table for all batches
                batch["pixel_values"],
                pixel_mean_A, # remove normalization
                pixel_std_A, # remove normalization
                pred_maps,
                masks,
                BATCH_INDEX, # correct indexing in table
            )
            BATCH_INDEX += 1
# Log table
wandb.log({f"{CURR_TASK}_TEST_AFTER_TRAINING_B": table})

# After compute the batches that were added are deleted
test_metrics_forgetting_A = metric.compute(
    num_labels=NUM_CLASSES, ignore_index=BG_VALUE_255, reduce_labels=False
)
mean_test_iou = test_metrics_forgetting_A["mean_iou"]
final_test_loss = test_running_loss / len(dataloaders[CURR_TASK]["test"].dataset)
wandb.log({
    f"Loss/test_latent_replay_32mb_{CURR_TASK}": final_test_loss,
    f"mIoU/test_latent_replay_32mb_{CURR_TASK}": mean_test_iou
})
print(f"Test Loss: {final_test_loss:.4f}, Test mIoU: {mean_test_iou:.4f}")


Test loop: 100%|██████████| 37/37 [00:40<00:00,  1.08s/it, loss=146.2014]


Test Loss: 29.2513, Test mIoU: 0.6448


In [ ]:
# Collect overall mIoU
mIoU_A_before = test_metrics_A["mean_iou"]
mIoU_B_before=test_metrics_B_before["mean_iou"]
mIoU_forgetting_A = test_metrics_forgetting_A["mean_iou"]
mIoU_B = test_metrics_B["mean_iou"]

# Collect per category mIoU
per_category_mIoU_A_before = np.array(test_metrics_A["per_category_iou"])
per_category_mIoU_forgetting_A = np.array(test_metrics_forgetting_A["per_category_iou"])
per_category_mIoU_B = np.array(test_metrics_B["per_category_iou"])
per_category_mIoU_B_before=np.array(test_metrics_B_before["per_category_iou"])

# Average learning accuracies (mIoUs)
avg_learning_acc = (mIoU_A_before + mIoU_B) / 2
per_category_avg_learning_acc = (per_category_mIoU_A_before + per_category_mIoU_B) / 2

# Forgetting
total_forgetting = mIoU_A_before - mIoU_forgetting_A
per_category_forgetting = (per_category_mIoU_A_before - per_category_mIoU_forgetting_A)

# Export evaluation metrics to WandB
wandb.log({
    "eval/avg_learning_acc": avg_learning_acc,
    "eval/total_forgetting": total_forgetting,
})

columns=["categories","per_category_mIoU_A_before","per_category_mIoU_B_before",
         "per_category_mIoU_B", "per_category_mIoU_forgetting_A",
         "per_category_avg_learning_acc","per_category_forgetting"]
data=[]

data.append(["background",per_category_mIoU_A_before[0],
                 per_category_mIoU_B_before[0],
                 per_category_mIoU_B[0],
                per_category_mIoU_forgetting_A[0],per_category_avg_learning_acc[0],
                per_category_forgetting[0]])

for cat_id in range(1,12):
    data.append([ZEISS_CATEGORIES[cat_id],per_category_mIoU_A_before[cat_id],
                 per_category_mIoU_B_before[cat_id],
                 per_category_mIoU_B[cat_id],
                per_category_mIoU_forgetting_A[cat_id],per_category_avg_learning_acc[cat_id],
                per_category_forgetting[cat_id]])
    
    
table = wandb.Table(columns=columns, data=data)
wandb.log({"per_category_metrics_table": table})

print("**** Overall mIoU ****")
print(f"mIoU on task A before training on B: {mIoU_A_before}")
print(f"mIoU on task B before training on B: {mIoU_B_before}")
print("\n")
print(f"mIoU on task B after training on B: {mIoU_B}")
print(f"mIoU on task A after training on B: {mIoU_forgetting_A}")

print("\n**** Per category mIoU ****")
print(f"Per category mIoU on task A before training on B: {per_category_mIoU_A_before}")
print(f"Per category mIoU on task B before training on B: {per_category_mIoU_B_before}")
print("\n")
print(f"Per category mIoU on task B after training on B: {per_category_mIoU_B}")
print(f"Per category mIoU on task A after training on B: {per_category_mIoU_forgetting_A}")

print("\n**** Average learning accuracies ****")
print(f"Average learning acc.: {avg_learning_acc}")
print(f"Per category Average learning acc.: {per_category_avg_learning_acc}")

print("\n**** Forgetting ****")
print(f"Total forgetting: {total_forgetting}")
print(f"Per category forgetting: {per_category_forgetting}")
wandb.finish()

if os.path.exists("artifacts/"):
    shutil.rmtree("artifacts/")

**** Overall mIoU ****
mIoU on task A before training on B: 0.74640957782497
mIoU on task B before training on B: 0.2716504918628598


mIoU on task B after training on B: 0.8015355734174626
mIoU on task A after training on B: 0.6447664778929338

**** Per category mIoU ****
Per category mIoU on task A before training on B: [0.97407127 0.90756186 0.78469834 0.60486195 0.57419152 0.43835761
 0.78412065 0.66920878 0.71826394 0.71686484 0.94148904 0.84322513]
Per category mIoU on task B before training on B: [0.9294207  0.345129   0.04438524 0.09931559 0.11231308 0.09308301
 0.04285594 0.08669283 0.04982049 0.15112783 0.77350991 0.53215228]


Per category mIoU on task B after training on B: [0.99141558 0.80730223 0.61140492 0.66703585 0.67010851 0.75876458
 0.83697675 0.78257037 0.82676566 0.79784037 0.96011946 0.9081226 ]
Per category mIoU on task A after training on B: [0.9716783  0.83684719 0.70311909 0.48163375 0.37241678 0.24903346
 0.66772254 0.37217511 0.61706323 0.68787911 0.9374360